In [ ]:
import os, csv
import rpy2.robjects as robjects
import plotly.graph_objects as go
import plotly.io

plotly.io.orca.config.executable='/usr/local/bin/orca'

%load_ext rpy2.ipython

In [ ]:
%%R
library(gplots);library(readr);library(ggplot2); library(DESeq2); library(dplyr); library(ggrepel)

In [ ]:
%%R
#Trimmed for no nam
params <- list(feature_counts='./merged_gene_counts.txt', 
               annotation='./metadata.csv', 
               condition='Type')
counts <- read.csv(params$feature_counts,
                       sep              = ",",
                       header           = TRUE, 
                       stringsAsFactors = FALSE, 
                       check.names      = FALSE);
rownames(counts) <- counts[,1]
counts <- counts[, -c(1:2)]
counts <- counts[ , order(names(counts))]
sampleTable <- read.csv(params$annotation, row.names = 1)
sampleTable <- sampleTable[ order(row.names(sampleTable)), ]
countdata <- as.matrix(counts)[, colnames(counts) %in% rownames(sampleTable)]
condition <- unlist(sampleTable[params$condition])
suppressMessages(library(DESeq2))
dds <- DESeqDataSetFromMatrix(countData=countdata, colData=sampleTable, design= as.formula(paste("~", params$condition)))
dds <- DESeq(dds)

rld <- rlogTransformation(dds)

suppressMessages(library(RColorBrewer))
(mycols <- brewer.pal(8, "Dark2")[1:length(unique(condition))])

data <- plotPCA(rld, intgroup=params$condition, returnData=TRUE)
rv = rowVars(assay(rld))
select = order(rv, decreasing = TRUE)[seq_len(min(500, length(rv)))]
pca = prcomp(t(assay(rld)[select, ]))

pc1var <- round(summary(pca)$importance[2,1]*100, digits=1)
pc2var <- round(summary(pca)$importance[2,2]*100, digits=1)
pc1lab <- paste0("PC1 (",as.character(pc1var),"%)")
pc2lab <- paste0("PC1 (",as.character(pc2var),"%)")

jpeg('./output.jpg')

p1 <- ggplot(data, aes(PC1, PC2))
p1 <- p1 + geom_point(aes(color=mycols[condition], shape=mycols[condition]),alpha=0.55,size=5)
p1 <- p1 + scale_shape_manual(values=c(15, 17, 19, 18))
p1 <- p1 + xlab(pc1lab)
p1 <- p1 + ylab(pc2lab)
p1 <- p1 + geom_text_repel(aes(label=rownames(data)))
p1 <- p1 + theme(legend.position = "none")
print(p1)
dev.off()